In [ ]:
%matplotlib inline

# Building a simple neural-network with Keras

Written by **Xavier Snelgrove** ([link to his GitHub](https://github.com/wxs/keras-mnist-tutorial/blob/master/MNIST%20in%20Keras.ipynb)), with minor modifications from **Rafael Padilha**.

This is a simple tutorial on working with neural networks in Keras. In this exercise, we will define, train and evaluate a simple feed-forward NN to classify handwritten digits.


## Imports
First let's import some prerequisites:

In [ ]:
import numpy as np
from random import shuffle, seed
seed(42)
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15,15) # Make the figures a bit bigger

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
from keras import optimizers

## Load training data

In [ ]:
nb_classes = 10

# the data, shuffled and split between trainVal and test sets
(trainVal_data, trainVal_label), (X_test, y_test) = mnist.load_data()

# We want now to split the trainVal data into train and validation sets
nData = trainVal_data.shape[0]  #find the size of trainVal
nTrain = int(nData * 0.8)  #80% to train, 20% to val

randomIdx = list(range(nData))   #randomly select indexes
shuffle(randomIdx)
trainIdx = randomIdx[:nTrain] 
valIdx = randomIdx[nTrain:]

# Split the data
X_val, y_val = trainVal_data[valIdx], trainVal_label[valIdx]
X_train, y_train = trainVal_data[trainIdx], trainVal_label[trainIdx]

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape), "\n"

print("X_val original shape", X_val.shape)
print("y_val original shape", y_val.shape), "\n"

print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

Let's look at some examples of the training data:

In [ ]:
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(X_train[i], cmap='gray', interpolation='none')
    plt.title("Class {}".format(y_train[i]))

## How balanced is our training?
Let's check how many examples we have for each class and calculate a class weight. This can be useful to tell the model to "pay more attention" during training to samples from an under-represented class.

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
print ("Train ---> ", dict(zip(unique, counts)), "\n")


from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', np.unique(y_train), y_train)
train_class_weights = dict(enumerate(class_weights))
print (train_class_weights, "\n")

In [ ]:
#For the test set ---> We will use this to find the normalized accuracy
unique, counts = np.unique(y_test, return_counts=True)
print("Test ---> ", dict(zip(unique, counts)), "\n")

test_sample_per_class = counts

## Format the data for training
Our neural-network is going to take a single vector for each training example, so we need to reshape the input so that each 28x28 image becomes a single 784 dimensional vector. We'll also scale the inputs to be in the range [0-1] rather than [0-255]

In [ ]:
X_train = X_train.reshape(48000, 784)
X_val = X_val.reshape(12000, 784)
X_test = X_test.reshape(10000, 784)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_val /= 255
X_test /= 255

print("Training matrix shape", X_train.shape)
print("Validation matrix shape", X_val.shape)
print("Testing matrix shape", X_test.shape)

Let's also modify the target matrices to be in the one-hot format, i.e.

```
0 -> [1, 0, 0, 0, 0, 0, 0, 0, 0]
1 -> [0, 1, 0, 0, 0, 0, 0, 0, 0]
2 -> [0, 0, 1, 0, 0, 0, 0, 0, 0]
3 -> [0, 0, 0, 1, 0, 0, 0, 0, 0]
...
9 -> [0, 0, 0, 0, 0, 0, 0, 0, 1]
```

Keras already has a method to do this, `np_utils.to_categorical`:

In [ ]:
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

# Build the neural network
Build the neural-network. Here we'll do a simple neural network with 4 layers (input + 2 hidden layers + output), using fully connected layers. In this type of layer, each neuron is connected to every input units. If we consider the first hidden layer, each of its neurons is connected to each pixel in the input image.
<img src="figure.png" />

In [ ]:
# The Sequential model is a linear stack of layers.
model = Sequential()

# Adding the first hidden layer with 512 nodes. We need to tell our model the input dimension
#in our case it is an array of length 784.
model.add(Dense(512, input_shape=(784,)))


# An "activation" is just a non-linear function applied to the output
# of the layer above. Here, we will apply the sigmoid activation 
model.add(Activation('sigmoid')) 
                              
                           
# Now we add the second hidden layer 
model.add(Dense(512))
model.add(Activation('sigmoid'))

# And connect it with the output layer, which has neurons equal to the number of classes
#in our problem (10 neurons as it will predict one-hot vectors of size 10)
model.add(Dense(10))

# Finally, we will use the softmax activation
model.add(Activation('softmax')) # "softmax", among other things, ensures the output 
                                 # is a valid probaility distribution, that is
                                 # that its values are all non-negative and sum to 1.

## Compile the model
Keras is built on top of TensorFlow or Theano, both packages that allow you to define a *computation graph* in Python, which they then compile and run efficiently on the CPU or GPU without the overhead of the Python interpreter.

When compiing a model, Keras asks you to specify your **loss function**, your **optimizer** and some **metrics**. The loss function we'll use here is called *categorical crossentropy*, and is a loss function well-suited to comparing two probability distributions.

Here our predictions are probability distributions across the ten different digits (e.g. "we're 80% confident this image is a 3, 10% sure it's an 8, 5% it's a 2, etc."), and the target is a probability distribution with 100% for the correct category, and 0 for everything else. The cross-entropy is a measure of how different your predicted distribution is from the target distribution. [More detail at Wikipedia](https://en.wikipedia.org/wiki/Cross_entropy)

The [optimizer](https://keras.io/optimizers) helps determine how quickly the model learns, how resistent it is to getting "stuck" or "blowing up". In this exercise we will use a variation of the Gradient Descent, called **Stocastic Gradient Descent (SGD)**. Instead of considering the whole training data at each iteration of the optimization, SGD only uses a batch sampled from the available training data at a time.

Besides that, we can define a number of metrics (such as accuracy, mean squared error, mean absolute error etc) to follow during training. **However, by default Keras does not output the normalized accuracy!**

In [ ]:
sgd = optimizers.SGD(lr=0.01) #lr = learning rate
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

## Train the model!
This is the fun part: you can feed the training data loaded in earlier into this model and it will learn to classify digits. 

But first, let's understand what are these arguments:
- **X_train & Y_train:** the training data and labels;
- **batch_size:** instead of optimizing your network on all training data, we usually optimize it over batches (small samples from the training data). In this case, we will update our weights (backpropagation) after forwarding/processing 128 images;
- **nb_epoch:** an epoch occurs when all your training data is processed/forwarded once. We are saying we will process our data 8 times;
- **verbose:** show loss after each epoch;
- **class_weight:** the weight of each class, considering they are not balanced. This will ponder the loss based on how frequent each class is;
- **validation_data:** after each epoch our model will predict these data and output the loss and accuracy.

In [ ]:
model.fit(X_train, Y_train,
          batch_size=128, epochs=8, verbose=1,
          class_weight = train_class_weights, 
          validation_data=(X_val, Y_val))

## Finally, evaluate its performance on the test data

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy (NOT NORMALIZED):', score[1])

## Normalized Accuracy

In [ ]:
# The predict_classes function outputs the highest probability class
# according to the trained classifier for each input example.
predicted_classes = model.predict_classes(X_test)

#Get the class for each one_hot array of our test samples
Y_test_classes = np.argmax(Y_test, axis=-1)

accPerClass = []
for classIdx in range(nb_classes):
    idx = (Y_test_classes == classIdx)
    
    correctPred = np.sum(predicted_classes[idx] == Y_test_classes[idx])
    accPerClass.append( correctPred / float(test_sample_per_class[classIdx]))
    
print (dict(zip(range(nb_classes),accPerClass)), "\n")
print ("Normalized Acc --> ", np.mean(accPerClass))

### Inspecting the output

It's always a good idea to inspect the output and make sure everything looks sane. Here we'll look at some examples it gets right, and some examples it gets wrong.

In [ ]:
# The predict_classes function outputs the highest probability class
# according to the trained classifier for each input example.
predicted_classes = model.predict_classes(X_test)

# Check which items we got right / wrong
correct_indices = np.nonzero(predicted_classes == y_test)[0]
incorrect_indices = np.nonzero(predicted_classes != y_test)[0]

In [ ]:
plt.figure()
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], y_test[correct]))
    
plt.figure()
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], y_test[incorrect]))

# References

There are lots of other great examples at the Keras homepage at http://keras.io and in the source code at https://github.com/fchollet/keras. Besides those, this notebook was adapted from [this](https://github.com/wxs/keras-mnist-tutorial/blob/master/MNIST%20in%20Keras.ipynb).